## Intrinsic Neuromodulation of Cognitive Affect (INCA)

***

### Brain States <i> vs </i> Psychological Variables 

* _This notebook uses brain data from the the Intrinsic Neuromodulation (IN) task/experiment to predict behavioral characteristics using LASSO_

* <b>Volitional Recall</b> (VR) [Valence, Arousal] <i> <u>vs</u> </i> <b>Psychological Variables</b> [DERS, CTQ, ...]

<i> Bush, K.A. et al. Common Functional Brain States Encode both Perceived Emotion and the Psychophysiological Response to Affective Stimuli. Sci Rep 8, 15444 (2018). https://doi.org/10.1038/s41598-018-33621-6</i>

***

### Optimization Objective

$ ( \; \frac{1}{2n_{samples}} \;) \; ||\;y - X\beta\;||^2_2 + \alpha\;||\; \beta \;||_1 $

### Optimization Objective Variables

| Variable | Description |
|:------|:-----------|
| $ α ≥ 0 $ | Regularization parameter: Modulates level of shrinkage | 
| $ β $ | LASSO estimator: Regression coefficients |

***

### Import Python libraries

In [ ]:
%matplotlib inline 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from scipy import stats
from scipy.stats import zscore
from sklearn.preprocessing import scale 
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso, LassoCV 
from sklearn.metrics import mean_squared_error
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

### Load and preprocess data

In [ ]:
##-- Load data
df=pd.read_csv('INCA_data.csv',index_col=None)

##-- Parse variables | static_vars = all psychometric variables
all_vars, static_vars = df.iloc[:,5:50], df.iloc[:,10:50]

##-- CTQ -> Trauma
ctq_vars = df.iloc[:,12:18]
##-- DERS -> Emotion Regulation
ders_vars = df.iloc[:,18:24]
##-- Personality scale
personality_vars = df.iloc[:,25:45]
##-- Demographics
demographic_vars = df.iloc[:,6:10]

##-- Sums of scales
df_sums=df.filter(items=['v_sig','a_sig',"IQ","BDI","CTQ",'DERS','STAI','PIES','EHI'])

##-- Parse VR values
V_VR, A_VR = [df[['v_sig']]], [df[['a_sig']]]

### Initialize LASSO functions
* Perform LASSO regressions to analyze how different features influence a target variable, and it does so for multiple groups of features. It also identifies the optimal regularization strength (alpha) via cross-validation.

`lasso` function
* Takes input features X and target variable y
* Generates alpha values for Lasso regularization
* Splits data into training and test sets
* Trains Lasso models with different alpha values, stores coefficients
* Plots coefficients against alpha values
* Finds the optimal alpha value using LassoCV (cross-validation)
* Refits the Lasso model using the optimal alpha
* Computes and prints R2 score, mean squared error, and coefficients

`lasso_IN` function
* Takes a group of input features X_group, a target variable y_predictor, and psychometric data static_vars
* For each group of variables in X_group, it concatenates them with static_vars and calls the lasso function

In [ ]:
def lasso(X,y):
    
    ##-- Generate alphas
    alphas = 10**np.linspace(1,-2,36)*0.5
    
    ##-- Split train & test data
    X_train, X_test , y_train, y_test = \
    train_test_split(X, y, test_size=.5, random_state=42)
    
    ##-- Train + plot LASSO model
    lasso = Lasso(max_iter=100000)
    coefs = [] 

    for a in alphas: 
        lasso.set_params(alpha=a) 
        lasso.fit(X_train, y_train)
        coefs.append(lasso.coef_)

    ax = plt.gca()
    ax.plot(alphas, coefs)
    ax.set_xscale('log')   
    plt.axis('tight') 
    plt.xlabel(r'alpha $\alpha$') 
    plt.ylabel('weights (coefs)')
    plt.title('y = %s | x = static_vars + %s' % (y.name,X.columns[0]))
    ax.legend(labels=pd.Series(X.columns), \
              loc=0, bbox_to_anchor=(1.3, .75)) 
    plt.show()

    ##-- Select the best alpha via cross validation (CV)
    lassocv = LassoCV(cv=18, max_iter=100000) 
    params = lassocv.fit(X_train, y_train)

    ##-- Refit LASSO model with optimal alpha
    ref_lasso_cv = lasso.set_params(alpha=lassocv.alpha_) 
    ref_params = lasso.fit(X_train, y_train) 
    
    ##-- Determine R2 score
    y_pred_lasso = lasso.fit(X_train, y_train).predict(X_test)
    r2_score_lasso = r2_score(y_test, y_pred_lasso)
    print("r^2 on test data : %f\n" % r2_score_lasso)

    ##-- Compute error
    mse = mean_squared_error(y_test, lasso.predict(X_test))

    ##-- Determine CV alpha
    cv_alpha = lassocv.alpha_
    print('MSE: %f \n\nCV alpha: %f \n\nCV params: \n\n%s\n' % (mse, cv_alpha, params))
    
    ##-- LASSO parameters
    print('LASSO params: \n\n%s\n' % ref_params)

    ##-- Coefficient estimates
    ce = pd.Series(lasso.coef_, index=X.columns)
    print('LASSO coefficient estimates: \n%s' % ce)
    
    ##-- OPTIONAL: Sparse coefficient estimates
    sce = pd.Series(lasso.sparse_coef_, index=X.columns)
    print('lasso sparse coefficient estimates: \n%s' % sce)     
    
    return

In [ ]:
def lasso_IN(X_group,y_predictor,static_vars):
    for var in X_group:
        y_var = y_predictor
        X_var = pd.concat([var,static_vars], axis=1)
        lasso(X_var,y_var)
    return

In [ ]:
##-- valence brain states (IN fMRI values - SVM output) vs cog-behav variables
lasso_IN(A_VR,df['v_sig'],static_vars)

In [ ]:
##-- arousal brain states (IN fMRI values - SVM output) vs cog-behav variables
lasso_IN(V_VR,df['a_sig'],static_vars)